# Instalar o tensorflow

In [1]:
!pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=1396d28338fff9769e98d0d55f05e17df7ce37a46d686c0109ffb872ee64e6f2
  Stored in directory: c:\users\joaop\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor


In [162]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import scipy.io
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor

# 1. Recolher os dados

In [144]:
def get_data_from_mat(train_file,test_file):
    train_mat = scipy.io.loadmat(train_file) 
    test_mat = scipy.io.loadmat(test_file) 
    print(train_mat.keys())
    print(test_mat.keys())
    train_np = np.array(train_mat['train_data']).transpose(2,0,1)
    test_np = np.array(test_mat['test_data']).transpose(2,0,1)
    print('Train shape: ',train_np.shape)
    print('Test shape: ',test_np.shape)
    return  train_np, test_np


train_data, test_data = get_data_from_mat('train_data.mat','test_data.mat')

dict_keys(['__header__', '__version__', '__globals__', 'train_data'])
dict_keys(['__header__', '__version__', '__globals__', 'test_data'])
Train shape:  (112, 90, 90)
Test shape:  (28, 90, 90)


In [145]:
train_data[0].shape

(90, 90)

In [146]:
train_data[0]

array([[0.        , 0.        , 0.13152513, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.13152513, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.28060623],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.28060623, 0.        ,
        0.        ]])

# 2. Preparar os dados

## 2.1. Dataset de treino

In [147]:
#Tornar cada matriz num vetor de uma dimensão (passar de 90x90 para 8100) e armazenar todos os valores de cada matriz
#O dicionário utilizado armazena, todos os valores com indíce X, pertencentes às diferentes 112 matrizes, sendo esse índice
#a chave do dic.
#Por exemplo: dictAttributes[0] contém o primeiro elemento de cada vetor anteriormente criado
dictAttributes = dict()
for i in range(len(train_data)):# len(train_data) - 112
    flatten_data = train_data[i].flatten()
    for j in range(len(flatten_data)):
        dictAttributes.setdefault(j,list())
        dictAttributes[j].append(flatten_data[j])
len(dictAttributes)

8100

In [148]:
#Nesta fase criam-se colunas com todos os elementos que estão numa dada posição do vetor, a label da coluna é essa posição
scans_series = []
for k in dictAttributes.keys():
    scans_series.append(pd.Series(dictAttributes.get(k)))

In [149]:
train_data_series = pd.concat(scans_series,axis=1)
train_data_series

,0,1,2,3,4,5,6,7,8,9,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,0.0,0.0,0.131525,0.0,0.0,0.0,0.271538,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.585006,0.0,0.280606,0.0,0.0
1,0.0,0.0,0.117704,0.0,0.0,0.0,0.316890,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.599437,0.0,0.207478,0.0,0.0
2,0.0,0.0,0.155806,0.0,0.0,0.0,0.348072,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.630374,0.0,0.246472,0.0,0.0
3,0.0,0.0,0.145214,0.0,0.0,0.0,0.310270,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.562505,0.0,0.287888,0.0,0.0
4,0.0,0.0,0.161360,0.0,0.0,0.0,0.385560,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.564043,0.0,0.234284,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,0.0,0.0,0.215767,0.0,0.0,0.0,0.390332,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.011150,0.0,0.660292,0.0,0.302675,0.0,0.0
108,0.0,0.0,0.143005,0.0,0.0,0.0,0.306380,0.0,0.0,0.0,...,0.0,0.011171,0.0,0.019141,0.0,0.630415,0.0,0.204125,0.0,0.0
109,0.0,0.0,0.116085,0.0,0.0,0.0,0.290941,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.574355,0.0,0.246784,0.0,0.0
110,0.0,0.0,0.156654,0.0,0.0,0.0,0.357540,0.0,0.0,0.0,...,0.0,0.010225,0.0,0.000000,0.0,0.631057,0.0,0.216259,0.0,0.0


In [150]:
TRAIN_CSV_PATH = 'train.csv'

In [151]:
rest_train_df = pd.read_csv(TRAIN_CSV_PATH)
rest_train_df.head(10) #Tem 112 linhas

,id,age,sex,education
0,1,13,1,7
1,2,14,0,8
2,3,15,1,9
3,4,15,1,9
4,5,15,1,9
5,6,15,1,9
6,7,16,0,10
7,8,16,1,10
8,9,16,0,10
9,10,17,0,11


In [152]:
targets_train = rest_train_df['age']
targets_train

0      13
1      14
2      15
3      15
4      15
       ..
107    77
108    67
109    55
110    76
111    69
Name: age, Length: 112, dtype: int64

In [153]:
train_data_series = pd.concat([targets_train] + scans_series,axis=1).sample(frac=1.0, random_state=2022).reset_index(drop=True)
train_data_series

,age,0,1,2,3,4,5,6,7,8,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,13,0.0,0.0,0.131525,0.0,0.0,0.0,0.271538,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.585006,0.0,0.280606,0.0,0.0
1,14,0.0,0.0,0.117704,0.0,0.0,0.0,0.316890,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.599437,0.0,0.207478,0.0,0.0
2,15,0.0,0.0,0.155806,0.0,0.0,0.0,0.348072,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.630374,0.0,0.246472,0.0,0.0
3,15,0.0,0.0,0.145214,0.0,0.0,0.0,0.310270,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.562505,0.0,0.287888,0.0,0.0
4,15,0.0,0.0,0.161360,0.0,0.0,0.0,0.385560,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.564043,0.0,0.234284,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,77,0.0,0.0,0.215767,0.0,0.0,0.0,0.390332,0.0,0.0,...,0.0,0.000000,0.0,0.011150,0.0,0.660292,0.0,0.302675,0.0,0.0
108,67,0.0,0.0,0.143005,0.0,0.0,0.0,0.306380,0.0,0.0,...,0.0,0.011171,0.0,0.019141,0.0,0.630415,0.0,0.204125,0.0,0.0
109,55,0.0,0.0,0.116085,0.0,0.0,0.0,0.290941,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.574355,0.0,0.246784,0.0,0.0
110,76,0.0,0.0,0.156654,0.0,0.0,0.0,0.357540,0.0,0.0,...,0.0,0.010225,0.0,0.000000,0.0,0.631057,0.0,0.216259,0.0,0.0


In [154]:
train_df_all = pd.DataFrame(train_data_series)
train_df_all

,age,0,1,2,3,4,5,6,7,8,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,13,0.0,0.0,0.131525,0.0,0.0,0.0,0.271538,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.585006,0.0,0.280606,0.0,0.0
1,14,0.0,0.0,0.117704,0.0,0.0,0.0,0.316890,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.599437,0.0,0.207478,0.0,0.0
2,15,0.0,0.0,0.155806,0.0,0.0,0.0,0.348072,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.630374,0.0,0.246472,0.0,0.0
3,15,0.0,0.0,0.145214,0.0,0.0,0.0,0.310270,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.562505,0.0,0.287888,0.0,0.0
4,15,0.0,0.0,0.161360,0.0,0.0,0.0,0.385560,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.564043,0.0,0.234284,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,77,0.0,0.0,0.215767,0.0,0.0,0.0,0.390332,0.0,0.0,...,0.0,0.000000,0.0,0.011150,0.0,0.660292,0.0,0.302675,0.0,0.0
108,67,0.0,0.0,0.143005,0.0,0.0,0.0,0.306380,0.0,0.0,...,0.0,0.011171,0.0,0.019141,0.0,0.630415,0.0,0.204125,0.0,0.0
109,55,0.0,0.0,0.116085,0.0,0.0,0.0,0.290941,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.574355,0.0,0.246784,0.0,0.0
110,76,0.0,0.0,0.156654,0.0,0.0,0.0,0.357540,0.0,0.0,...,0.0,0.010225,0.0,0.000000,0.0,0.631057,0.0,0.216259,0.0,0.0


## 2.2. Dataset de validação

In [156]:
train_df, val_df = train_test_split(train_df_all, train_size = 0.7, shuffle=True, random_state=2022)
print('Quantidade de casos para treino: ', len(train_df))
print('Quantidade de casos para validação: ', len(val_df))

Quantidade de casos para treino:  78
Quantidade de casos para validação:  34


In [159]:
class MyDataset(Dataset):
    def __init__(self,train_df,validation_df):
        self.x_train = train_df.values[:,1:]
        self.y_train = train_df.values[:,0]
        self.x_val = validation_df.values[:,1:]
        self.y_val = validation_df.values[:,0]
        
        self.x_train = self.x_train.astype('float32')
        self.x_val = self.x_val.astype('float32')
        self.y_train = self.y_train.astype('long')
        self.y_val = self.y_val.astype('long')
        
    def __len_train__(self):
        return len(self.x_train)
    
    def __len_test__(self):
        return len(self.x_val)
    
    def __getitem_train__(self, idx):
        return [self.x_train[idx], self.y_train[idx]]
    
    def __getitem_test__(self, idx):
        return [self.x_val[idx], self.y_val[idx]]
    
    def get_splits_flat(self):
        x_train = torch.from_numpy(np.array(self.x_train))
        y_train = torch.from_numpy(np.array(self.y_train))
        x_val = torch.from_numpy(np.array(self.x_val))
        y_val = torch.from_numpy(np.array(self.y_val))
        train = torch.utils.data.TensorDataset(x_train,y_train)
        val = torch.utils.data.TensorDataset(x_val,y_val)
        return train, val

In [161]:
def prepare_data_flat(df_train, df_validation):
    dataset = MyDataset(df_train, df_validation)
    train, validation = dataset.get_splits_flat()
    
    train_dl = DataLoader(train, batch_size = BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(validation, batch_size = BATCH_SIZE, shuffle=True)
    train_dl_all = DataLoader(train, batch_size = len(train), shuffle=False)
    val_dl_all = DataLoader(validation, batch_size = len(validation), shuffle=False)
    return train_dl, val_dl, train_dl_all, val_dl_all

train_dl, val_dl, train_dl_all, val_dl_all = prepare_data_flat(train_df, val_df)

NameError: name 'torch' is not defined

# 3. Treinar o modelo

NameError: name 'train_generator' is not defined